# Data access in SWISS MODEL Repository


SMR currently provide models for the Swiss-Prot section of UniProtKB and for the reference proteomes of the following model organisms: H. sapiens, M. musculus, C. elegans, E. coli K12, A. thaliana, D. melanogaster, S. cerevisiae, Caulobacter crescentus, M. tuberculosis, P. aeruginosa, Staphylococcus aureus, and P. falciparum.

In [133]:
#import dependencies
import pandas as pd
import json
import seaborn as sns
import matplotlib.pyplot as plt

## Two modes for bulk data access:
1. Download models for reference proteome (13 model organisms)
2. Use API services to access data for specific UniprotKB sequence



### 1. Download models for reference proteome (13 model organisms)

Reference proteome: Homo sapiens. Metadata file (models and structures) downloaded from [SMR](https://swissmodel.expasy.org/repository/) 

In [2]:
#open file downloaded from SMR
with open('metadata/SWISS-MODEL_Repository/INDEX.json', 'r') as file:
     variable_name = json.load(file)

In [142]:
#display the response object
# variable_name

In [92]:
index = variable_name['index']
len(index)

155644

In [93]:
# Convert to a dataframe
df = pd.DataFrame(index)
df.head()

,uniprot_ac,iso_id,uniprot_seq_length,uniprot_seq_md5,coordinate_id,provider,from,to,template,qmean,qmean_norm,seqid,url
0,P31946,P31946-1,246,c82f2efd57f939ee3c4e571708dd31a8,6041e7c2a2248c526cf471a1,SWISSMODEL,3,232,2c1j.1.A,-0.614406,0.749879,86.8852,https://swissmodel.expasy.org/repository/unipr...
1,P31946-2,P31946-2,244,0bd136ab4d2a4ee6a9caf7e4c0ffc256,6041f0521cd80f52a4c27cf7,SWISSMODEL,1,232,1qjb.1.A,-0.882766,0.741682,87.2428,https://swissmodel.expasy.org/repository/unipr...
2,P31946,P31946-1,246,c82f2efd57f939ee3c4e571708dd31a8,5f3dae76676e7622ac974741,PDB,3,233,6gnj,NaN,NaN,,https://swissmodel.expasy.org/repository/unipr...
3,P31946,P31946-1,246,c82f2efd57f939ee3c4e571708dd31a8,5f3da464676e7622ac967bee,PDB,3,232,6gnn,NaN,NaN,,https://swissmodel.expasy.org/repository/unipr...
4,P31946,P31946-1,246,c82f2efd57f939ee3c4e571708dd31a8,5f3d9c4a676e7622ac95d5f2,PDB,3,233,5n10,NaN,NaN,,https://swissmodel.expasy.org/repository/unipr...


Data contains structures from Swiss Model as well as PDB. We will have to filter out PDB.

In [94]:
#Check for data summary, missing values, datatypes etc.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155644 entries, 0 to 155643
Data columns (total 13 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   uniprot_ac          155644 non-null  object 
 1   iso_id              110507 non-null  object 
 2   uniprot_seq_length  155644 non-null  int64  
 3   uniprot_seq_md5     155644 non-null  object 
 4   coordinate_id       155644 non-null  object 
 5   provider            155644 non-null  object 
 6   from                155644 non-null  int64  
 7   to                  155644 non-null  int64  
 8   template            155644 non-null  object 
 9   qmean               91463 non-null   float64
 10  qmean_norm          91463 non-null   object 
 11  seqid               155644 non-null  object 
 12  url                 155644 non-null  object 
dtypes: float64(1), int64(3), object(9)
memory usage: 15.4+ MB


Different data types, missing values in different columns. Filter out PDB and recheck.

In [98]:
# Check unique providers 
df["provider"].unique()

array(['SWISSMODEL', 'PDB'], dtype=object)

In [99]:
# Filter models from SMR
df_swissmodel = df.loc[(df['provider'] =="SWISSMODEL")]
len(df_swissmodel)

91463

In [101]:
df_swissmodel.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 91463 entries, 0 to 155643
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   uniprot_ac          91463 non-null  object 
 1   iso_id              73787 non-null  object 
 2   uniprot_seq_length  91463 non-null  int64  
 3   uniprot_seq_md5     91463 non-null  object 
 4   coordinate_id       91463 non-null  object 
 5   provider            91463 non-null  object 
 6   from                91463 non-null  int64  
 7   to                  91463 non-null  int64  
 8   template            91463 non-null  object 
 9   qmean               91463 non-null  float64
 10  qmean_norm          91463 non-null  object 
 11  seqid               91463 non-null  object 
 12  url                 91463 non-null  object 
dtypes: float64(1), int64(3), object(9)
memory usage: 9.8+ MB


Total 91463 models are available for homo sapiens. 
No more missing values except for 'iso_id' represents absence of isoforms. 

Note to self: Trying to change object type to float throws 'ValueError' for this df but works fine with 'uniprot' df that follows later. Need to check and debug.

### Metadata definition 

In [7]:
pd.DataFrame(df.columns)

,0
0,uniprot_ac
1,iso_id
2,uniprot_seq_length
3,uniprot_seq_md5
4,coordinate_id
5,provider
6,from
7,to
8,template
9,qmean


## Number of unique sequences modeled

In [106]:
# Check for unique Uniprot acc ids, how many templates/models are available, their scores etc.
df_swissmodel["uniprot_ac"].value_counts()

Q02388-2    30
Q02388      30
A6NN14      30
O43345-2    29
P58107      28
            ..
O14829-3     1
A8MTT3       1
Q9UKI3       1
P30556       1
Q96A00-2     1
Name: uniprot_ac, Length: 36891, dtype: int64

Models available for 36891 unique sequences

## Instances of multiple models

### Case 1: No isoform

In [107]:
#Check multiple instances of a Sequence
uniprot = df_swissmodel.loc[df_swissmodel["uniprot_ac"]=='A6NN14']
uniprot

,uniprot_ac,iso_id,uniprot_seq_length,uniprot_seq_md5,coordinate_id,provider,from,to,template,qmean,qmean_norm,seqid,url
153170,A6NN14,NaN,1252,ea25bb254fdb4e006a3c96382f788da4,6046a2ad57396bb9ff433ea1,SWISSMODEL,655,931,5v3m.1.C,-2.557852,0.671339,50.974,https://swissmodel.expasy.org/repository/unipr...
153171,A6NN14,NaN,1252,ea25bb254fdb4e006a3c96382f788da4,6046a2ac57396bb9ff433e7d,SWISSMODEL,851,1155,5v3m.1.C,-3.216336,0.657585,51.6234,https://swissmodel.expasy.org/repository/unipr...
153172,A6NN14,NaN,1252,ea25bb254fdb4e006a3c96382f788da4,6046a2ad57396bb9ff433e95,SWISSMODEL,373,651,5wjq.1.C,-2.995696,0.654303,53.7367,https://swissmodel.expasy.org/repository/unipr...
153173,A6NN14,NaN,1252,ea25bb254fdb4e006a3c96382f788da4,6046a2ad57396bb9ff433e8f,SWISSMODEL,263,539,5v3m.1.C,-2.768441,0.662862,51.6234,https://swissmodel.expasy.org/repository/unipr...
153174,A6NN14,NaN,1252,ea25bb254fdb4e006a3c96382f788da4,6046a2ad57396bb9ff433e9b,SWISSMODEL,195,455,5v3m.1.C,-2.698588,0.664927,48.9726,https://swissmodel.expasy.org/repository/unipr...
153175,A6NN14,NaN,1252,ea25bb254fdb4e006a3c96382f788da4,6046a2ac57396bb9ff433e6b,SWISSMODEL,739,1015,5v3m.1.C,-2.902971,0.657447,51.2987,https://swissmodel.expasy.org/repository/unipr...
153176,A6NN14,NaN,1252,ea25bb254fdb4e006a3c96382f788da4,6046a2ac57396bb9ff433e89,SWISSMODEL,317,595,5wjq.1.C,-2.622521,0.669244,53.0249,https://swissmodel.expasy.org/repository/unipr...
153177,A6NN14,NaN,1252,ea25bb254fdb4e006a3c96382f788da4,6046a2ad57396bb9ff433ea7,SWISSMODEL,459,735,5v3m.1.C,-2.482402,0.674376,52.2727,https://swissmodel.expasy.org/repository/unipr...
153178,A6NN14,NaN,1252,ea25bb254fdb4e006a3c96382f788da4,6046a2ac57396bb9ff433e4d,SWISSMODEL,515,791,5v3m.1.C,-2.858754,0.659227,52.2727,https://swissmodel.expasy.org/repository/unipr...
153179,A6NN14,NaN,1252,ea25bb254fdb4e006a3c96382f788da4,6046a2ad57396bb9ff433ead,SWISSMODEL,627,903,5v3m.1.C,-2.453341,0.675546,52.2727,https://swissmodel.expasy.org/repository/unipr...


[Zinc finger protein](https://swissmodel.expasy.org/repository/uniprot/A6NN14?template=5v3m.1.C&range=319-595): An example of a protein sequence that uses multiple templates for different coverage within the sequence

<img src="zinc_finger.png">





Observations:
No isoform for the sequence
Overlapping sequence range for a given template




In [108]:
uniprot[["qmean_norm", "seqid"]]= uniprot[["qmean_norm", "seqid"]].astype(float)

/Users/harmeet/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:3065: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [109]:
uniprot.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30 entries, 153170 to 153199
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   uniprot_ac          30 non-null     object 
 1   iso_id              0 non-null      object 
 2   uniprot_seq_length  30 non-null     int64  
 3   uniprot_seq_md5     30 non-null     object 
 4   coordinate_id       30 non-null     object 
 5   provider            30 non-null     object 
 6   from                30 non-null     int64  
 7   to                  30 non-null     int64  
 8   template            30 non-null     object 
 9   qmean               30 non-null     float64
 10  qmean_norm          30 non-null     float64
 11  seqid               30 non-null     float64
 12  url                 30 non-null     object 
dtypes: float64(3), int64(3), object(7)
memory usage: 3.3+ KB


### Check templates

In [50]:
# Number of models by each template
quality = uniprot.groupby('template').count()
quality

,uniprot_ac,iso_id,uniprot_seq_length,uniprot_seq_md5,coordinate_id,provider,from,to,qmean,qmean_norm,seqid,url
template,,,,,,,,,,,,
5v3m.1.C,26,0,26,26,26,26,26,26,26,26,26,26
5wjq.1.C,4,0,4,4,4,4,4,4,4,4,4,4


2 templates used, of which 26 models are generated using **5v3m.1.C** and 4 models are generated using **5wjq.1.C**

In [143]:
# Create dataframes to hold data for each template
v3m = uniprot.loc[(uniprot['template']=='5v3m.1.C')]
wjq = uniprot.loc[(uniprot['template']=='5wjq.1.C')]
# v3m.hist(figsize=(9, 9));

### Check the best models in terms of sequence identity, QMean and QMean_norm score

In [79]:
# Print out the best score for each template
print(f"Highest scoring model for Template 5v3m ")
print('-'*16)
print(f"Maximmum sequence identity is: {v3m['seqid'].max()}")
print(f"Highest qmean is: {v3m['qmean'].max()}")
print(f"Highest normalized qmean  is: {v3m['qmean_norm'].max()}")

print(f"Template 5wjq")
print('-'*16)
print(f"Maximmum sequence identity is: {wjq['seqid'].max()}")
print(f"Highest qmean is: {wjq['qmean'].max()}")
print(f"Highest normalized qmean  is: {wjq['qmean_norm'].max()}")

Highest scoring model for Template 5v3m 
----------------
Maximmum sequence identity is: 54.2207794189
Highest qmean is: -2.3711222689
Highest normalized qmean  is: 0.678855513
Template 5wjq
----------------
Maximmum sequence identity is: 53.9285697937
Highest qmean is: -2.6225212461
Highest normalized qmean  is: 0.6692444369


In [83]:
# Check if these values come from the same model for template 5v3m 
best_v3m=v3m.loc[((v3m['seqid']== 54.2207794189) | (v3m['qmean']== -2.3711222689) | (v3m['qmean_norm']== 0.678855513))]
best_v3m                

,uniprot_ac,iso_id,uniprot_seq_length,uniprot_seq_md5,coordinate_id,provider,from,to,template,qmean,qmean_norm,seqid,url
153180,A6NN14,NaN,1252,ea25bb254fdb4e006a3c96382f788da4,6046a2ac57396bb9ff433e71,SWISSMODEL,319,595,5v3m.1.C,-2.371122,0.678856,52.597404,https://swissmodel.expasy.org/repository/unipr...
153183,A6NN14,NaN,1252,ea25bb254fdb4e006a3c96382f788da4,6046a2ad57396bb9ff433eb3,SWISSMODEL,403,679,5v3m.1.C,-2.680020,0.666422,54.220779,https://swissmodel.expasy.org/repository/unipr...


There are two different models with best scores. The one with highest qmean and qmean_norm has a seq identity of 52.59%.
The one with the highest sequence identity i.e. 54% has a qmean of -2.6 and qmean_norm of 0.66. Both the models map to different but some overlapping sequence range in the target. Both provide a query coverage of 276 residues.

In [84]:
# Query coverage
best_v3m["coverage"]=best_v3m["to"]-best_v3m["from"]
best_v3m

<ipython-input-84-c60c1393cff4>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,uniprot_ac,iso_id,uniprot_seq_length,uniprot_seq_md5,coordinate_id,provider,from,to,template,qmean,qmean_norm,seqid,url,coverage
153180,A6NN14,NaN,1252,ea25bb254fdb4e006a3c96382f788da4,6046a2ac57396bb9ff433e71,SWISSMODEL,319,595,5v3m.1.C,-2.371122,0.678856,52.597404,https://swissmodel.expasy.org/repository/unipr...,276
153183,A6NN14,NaN,1252,ea25bb254fdb4e006a3c96382f788da4,6046a2ad57396bb9ff433eb3,SWISSMODEL,403,679,5v3m.1.C,-2.680020,0.666422,54.220779,https://swissmodel.expasy.org/repository/unipr...,276


In [88]:
# Check if these values come from the same model for template 5wjq
best_wjq = wjq.loc[((wjq['seqid']== 53.9285697937) | (wjq['qmean']== -2.6225212461) | (wjq['qmean_norm']== 0.6692444369)) ]
best_wjq

,uniprot_ac,iso_id,uniprot_seq_length,uniprot_seq_md5,coordinate_id,provider,from,to,template,qmean,qmean_norm,seqid,url
153176,A6NN14,NaN,1252,ea25bb254fdb4e006a3c96382f788da4,6046a2ac57396bb9ff433e89,SWISSMODEL,317,595,5wjq.1.C,-2.622521,0.669244,53.02491,https://swissmodel.expasy.org/repository/unipr...
153193,A6NN14,NaN,1252,ea25bb254fdb4e006a3c96382f788da4,6046a2ad57396bb9ff433eb9,SWISSMODEL,457,735,5wjq.1.C,-3.281607,0.642855,53.92857,https://swissmodel.expasy.org/repository/unipr...


In [89]:
# Query coverage
best_wjq["coverage"]=best_wjq["to"]-best_wjq["from"]
best_wjq

<ipython-input-89-d522b193da4e>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,uniprot_ac,iso_id,uniprot_seq_length,uniprot_seq_md5,coordinate_id,provider,from,to,template,qmean,qmean_norm,seqid,url,coverage
153176,A6NN14,NaN,1252,ea25bb254fdb4e006a3c96382f788da4,6046a2ac57396bb9ff433e89,SWISSMODEL,317,595,5wjq.1.C,-2.622521,0.669244,53.02491,https://swissmodel.expasy.org/repository/unipr...,278
153193,A6NN14,NaN,1252,ea25bb254fdb4e006a3c96382f788da4,6046a2ad57396bb9ff433eb9,SWISSMODEL,457,735,5wjq.1.C,-3.281607,0.642855,53.92857,https://swissmodel.expasy.org/repository/unipr...,278


## Case 2 : With isoform

In [116]:
#Check multiple instances of a Sequence
# Q02388-2    30
# Q02388      30
# A6NN14      30
# O43345-2    29
# P58107      28
case_iso = df_swissmodel.loc[df_swissmodel["uniprot_ac"]=='Q02388']
case_iso.head()

,uniprot_ac,iso_id,uniprot_seq_length,uniprot_seq_md5,coordinate_id,provider,from,to,template,qmean,qmean_norm,seqid,url
29459,Q02388,Q02388-1,2944,5fee08b99008bd3ad21b01cd193d49a7,604a3fbac2251a4a20f0bb74,SWISSMODEL,612,856,1fnh.1.A,-3.377565,0.636677,30.5085,https://swissmodel.expasy.org/repository/unipr...
29460,Q02388,Q02388-1,2944,5fee08b99008bd3ad21b01cd193d49a7,604a3fbac2251a4a20f0bb6a,SWISSMODEL,1051,1237,6s4c.1.A,0.876179,0.817581,79.4737,https://swissmodel.expasy.org/repository/unipr...
29461,Q02388,Q02388-1,2944,5fee08b99008bd3ad21b01cd193d49a7,604a3fbac2251a4a20f0bb62,SWISSMODEL,685,1028,3t1w.1.A,-4.398587,0.620282,26.8072,https://swissmodel.expasy.org/repository/unipr...
29462,Q02388,Q02388-1,2944,5fee08b99008bd3ad21b01cd193d49a7,604a3fbac2251a4a20f0bb7c,SWISSMODEL,829,1049,6mfa.1.A,-2.468128,0.68002,35.5769,https://swissmodel.expasy.org/repository/unipr...
29463,Q02388,Q02388-1,2944,5fee08b99008bd3ad21b01cd193d49a7,604a3fbac2251a4a20f0bb5e,SWISSMODEL,512,753,6mfa.1.A,-3.514206,0.632059,30.8017,https://swissmodel.expasy.org/repository/unipr...


In [120]:
len(case_iso)

30

### Check Templates

In [140]:
# Number of models by each template
templates = case_iso.groupby('template')['qmean'].max()
templates

template
1aox.1.A   -2.876988
1fnf.1.A   -2.054059
1fnh.1.A   -2.220108
1tdq.1.A   -3.219112
3r8q.1.A   -2.176573
3t1w.1.A   -2.579022
5e7l.1.A   -4.908399
5n47.1.B   -2.068055
5n47.3.B   -2.740505
6mfa.1.A   -2.156537
6s4c.1.A    0.876179
6tpw.1.A   -4.716267
Name: qmean, dtype: float64

12 templates used for generating 30 models

In [123]:
templates_list = case_iso["template"].unique()
templates_list

array(['1fnh.1.A', '6s4c.1.A', '3t1w.1.A', '6mfa.1.A', '1tdq.1.A',
       '1fnf.1.A', '6tpw.1.A', '5e7l.1.A', '3r8q.1.A', '1aox.1.A',
       '5n47.3.B', '5n47.1.B'], dtype=object)

In [124]:
case_iso[["qmean_norm", "seqid"]]= case_iso[["qmean_norm", "seqid"]].astype(float)
case_iso

/Users/harmeet/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:3065: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,uniprot_ac,iso_id,uniprot_seq_length,uniprot_seq_md5,coordinate_id,provider,from,to,template,qmean,qmean_norm,seqid,url
29459,Q02388,Q02388-1,2944,5fee08b99008bd3ad21b01cd193d49a7,604a3fbac2251a4a20f0bb74,SWISSMODEL,612,856,1fnh.1.A,-3.377565,0.636677,30.508474,https://swissmodel.expasy.org/repository/unipr...
29460,Q02388,Q02388-1,2944,5fee08b99008bd3ad21b01cd193d49a7,604a3fbac2251a4a20f0bb6a,SWISSMODEL,1051,1237,6s4c.1.A,0.876179,0.817581,79.473686,https://swissmodel.expasy.org/repository/unipr...
29461,Q02388,Q02388-1,2944,5fee08b99008bd3ad21b01cd193d49a7,604a3fbac2251a4a20f0bb62,SWISSMODEL,685,1028,3t1w.1.A,-4.398587,0.620282,26.807228,https://swissmodel.expasy.org/repository/unipr...
29462,Q02388,Q02388-1,2944,5fee08b99008bd3ad21b01cd193d49a7,604a3fbac2251a4a20f0bb7c,SWISSMODEL,829,1049,6mfa.1.A,-2.468128,0.680020,35.576923,https://swissmodel.expasy.org/repository/unipr...
29463,Q02388,Q02388-1,2944,5fee08b99008bd3ad21b01cd193d49a7,604a3fbac2251a4a20f0bb5e,SWISSMODEL,512,753,6mfa.1.A,-3.514206,0.632059,30.801687,https://swissmodel.expasy.org/repository/unipr...
29464,Q02388,Q02388-1,2944,5fee08b99008bd3ad21b01cd193d49a7,604a3fbac2251a4a20f0bb70,SWISSMODEL,231,575,6mfa.1.A,-2.156537,0.699437,34.055729,https://swissmodel.expasy.org/repository/unipr...
29465,Q02388,Q02388-1,2944,5fee08b99008bd3ad21b01cd193d49a7,604a3fbac2251a4a20f0bb5a,SWISSMODEL,634,846,1tdq.1.A,-3.989722,0.621604,26.570047,https://swissmodel.expasy.org/repository/unipr...
29466,Q02388,Q02388-1,2944,5fee08b99008bd3ad21b01cd193d49a7,604a3fb9c2251a4a20f0bb2e,SWISSMODEL,507,849,1fnf.1.A,-2.469547,0.688746,34.920635,https://swissmodel.expasy.org/repository/unipr...
29467,Q02388,Q02388-1,2944,5fee08b99008bd3ad21b01cd193d49a7,604a3fbac2251a4a20f0bb36,SWISSMODEL,694,978,6tpw.1.A,-4.716267,0.584988,30.909090,https://swissmodel.expasy.org/repository/unipr...
29468,Q02388,Q02388-1,2944,5fee08b99008bd3ad21b01cd193d49a7,604a3fbac2251a4a20f0bb4e,SWISSMODEL,343,677,1fnf.1.A,-2.967754,0.670422,30.491804,https://swissmodel.expasy.org/repository/unipr...


In [138]:
# bar plot for each template
subset = case_iso[['qmean', 'qmean_norm', 'seqid', 'template']]
subset.distplot(figsize=(9,9));


AttributeError: 'DataFrame' object has no attribute 'distplot'

In [ ]:
subset.hist(figsize=(9,9));

### 2. Use API services to access data for specific UniprotKB sequence


Endpoint construction: 

Base url = ‘https://swissmodel.expasy.org/repository/uniprot/’

param = AccessionID.json?provider=swissmodel


In [21]:
# API call returns list of models + experimental structures
import requests
url = "https://swissmodel.expasy.org/repository/uniprot/A6NN14.json?provider=swissmodel"
response = requests.get(url).json()
print(json.dumps(response, indent = 4, sort_keys = True))

{
    "api_version": "2.0",
    "query": {
        "ac": "A6NN14",
        "provider": [
            "swissmodel"
        ]
    },
    "query_date": "2021-04-06T19:39:34.015Z",
    "result": {
        "crc64": "DA92ED8A0114DE8F",
        "md5": "ea25bb254fdb4e006a3c96382f788da4",
        "sequence": "MPGAPGSLEMGPLTFRDVTIEFSLEEWQCLDTVQQNLYRDVMLENYRNLVFLGMAVFKPDLITCLKQGKEPWNMKRHEMVTKPPVMRSHFTQDLWPDQSTKDSFQEVILRTYARCGHKNLRLRKDCKSANEGKMHKEGYNKLNQCRTATQRKIFQCNKHMKVFHKYSNRNKVRHTKKKTFKCIKCSKSFFMLSCLIRHKRIHIRQNIYKCEERGKAFKSFSTLTKHKIIHTEDKPYKYKKCGNAFKFSSTFTKHKRIHTGETPFRCEECGKAFNQSSNLTDHKRIHTGEKTYKCEECGKAFKGSSNFNAHKVIHTAEKPYKCEDCGKTFNHFSALRKHKIIHTGKKPYKREECGKAFSQSSTLRKHEIIHTGEKPYKCEECGKAFKWSSKLTVHKVVHTGEKPYKCEECGKAFSQFSTLKKHKIIHTGKKPYKCEECGKAFNSSSTLMKHKIIHTGEKPYKCEECGKAFRQSSHLTRHKAIHTGEKPYKCEECGKAFNHFSDLRRHKIIHTGKKPYKCEECGKAFSQSSTLRNHQIIHTGEKPYKCEECGKAFKWSSKLTVHKVIHTGEKPCKCEECGKAFKHFSALRKHKVIHTREKLYKCEECGKAFNNSSILAKHKIIHTGKKPYKCEECGKAFRQSSHLTRHKAIHTGEKPYKCEECGKAFSHFSALRRHKIIHTGKKPYKCEECGKAFSHFSA

Other custom parameters:
    https://swissmodel.expasy.org/repository/uniprot/P0DTD1?template=6m71.1.A
        range=1024-1194
        from=1024&to=1194
        csm=969F65FCC0BC86FD (The CRC64 checksum of the protein sequence. This can be used to ensure the sequence for the given UniProtKB entry matches our record.)